In [33]:
import pandas as pd
from itertools import chain
from collections import Counter
from utils import load_json_config

In [34]:
config = load_json_config("config.json")

In [35]:
!gdown {config["raw_dataset"]}

Downloading...
From: https://drive.google.com/uc?id=15mXCMH9lpOGCk-z9MTDWx5Uuj6kkL5JL
To: /home/yehor/Desktop/Genre_based_on_album_cover/raw_data.csv
100%|████████████████████████████████████████| 206k/206k [00:00<00:00, 4.65MB/s]


NOTE: YOU HAVE TO ALTER THE PATH BELOW BASED ON WHEREVER YOU DOWNLOADED THE DATASET

In [36]:
df = pd.read_csv("/home/yehor/Desktop/Genre_based_on_album_cover/raw_data.csv")
df

,Unnamed: 0,album_index,album_id,genres
0,0,0,5n1GSzC1Reao29ScnpLYqp,"[['florida drill', 'florida rap', 'melodic rap..."
1,1,1,6UYZEYjpN1DYRW0kqFy9ZE,"[['hip hop', 'philly rap', 'pop rap', 'rap', '..."
2,2,2,7uVimUILdzSZG4KKKWToq0,"[['adult standards', 'canadian pop', 'jazz pop..."
3,3,3,35s58BRTGAEWztPo9WqCIs,[[]]
4,4,4,41GuZcammIkupMPKH2OJ6I,"[['rap', 'slap house']]"
...,...,...,...,...
2158,2158,2158,2I13eciqP8kdMa4kQQwauI,[[]]
2159,2159,2159,49lwR8y1i51wrQRnhw5C3H,"[['contemporary country', 'country', 'country ..."
2160,2160,2160,3q2tS3Qy39pOGDu3vsHQb5,"[['album rock', 'beatlesque', 'classic rock', ..."
2161,2161,2161,5wePyi4cqqTE9moAAXXbQu,"[['ccm', 'christian alternative rock', 'christ..."


In [37]:
maps = load_json_config("fewer_categories.json")

In [38]:
indexed_categories = maps["indexed_categories"]
keyword_to_category = maps["keyword_to_category"]

In [39]:
def extract_genres(genres):
    return list(chain.from_iterable(eval(genres)))

In [40]:
def parent_genre(sub_genre):
    if not sub_genre or not isinstance(sub_genre, str):
        return "unknown"
    
    sub_genre = sub_genre.lower()
    for keyword, category in keyword_to_category.items():
        if keyword in sub_genre:
            return category
    
    return "unknown"

In [41]:
def assign_genre(sub_genres):
    genre_counts = Counter([parent_genre(sub_genre) for sub_genre in sub_genres])
    if len(genre_counts) == 0:
        return "unknown"
    return max(genre_counts.items(), key=lambda item: item[1])[0]  # return the genre with max count

In [42]:
cleaned_df = pd.DataFrame({"album_id": pd.Series(dtype="int"), "genre": pd.Series(dtype="int")})
for row in df.itertuples(index=True, name="Row"):
    genre = assign_genre(extract_genres(row.genres))
    if genre == "unknown":
        continue
    new_row = pd.DataFrame([{"album_id": int(row.album_index), "genre": indexed_categories[genre]}])
    cleaned_df = pd.concat([cleaned_df, new_row], ignore_index=True)

cleaned_df

,album_id,genre
0,0,0
1,1,0
2,4,0
3,6,2
4,7,0
...,...,...
1292,2151,2
1293,2155,1
1294,2156,2
1295,2160,2


In [43]:
cleaned_df.to_csv(config["organized_dataset_name"], index=False)